# EDA on Titanic Data

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [86]:
df = pd.read_csv('./data/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [87]:
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [88]:
df.shape

(891, 12)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [90]:
# Types of Columns

# Numerical         Categorical         Target
# -----------------------------------------------
# Age               Embarked            Survived
# Fare              Sex
# Survived          Cabin
# Pclass            Ticket
# SibSp 
# Parch

## Feature Engineering and Selection

### Creating New Features (Feature Engineering)

In [91]:
# Creating clean_df 

clean_df = df.copy()

In [92]:
# SibSp + Parch = Family_size

clean_df['Family_size'] = clean_df['SibSp'] + clean_df['Parch'] + 1 

In [93]:
# Family_type
# 1 -> alone
# 2-4 -> small
# >5 -> large

def apply_family_size(num):
    
    if num == 1:
        return 'alone'
    elif num>1 and num<5:
        return 'small'
    else:
        return 'large'
    

In [94]:
clean_df['Family_type'] = clean_df['Family_size'].apply(apply_family_size)

### Encoding Categorical Variables

In [95]:
# Nominal Encoding on Sex Column (Nominal - Binary)

clean_df = pd.get_dummies(clean_df, columns=['Sex'], dtype=int)

# we can use drop_first+True it will drop Sex_female column and we can identify if the passenger is male or not(female).

In [96]:
# Nominal Encoding on Embarked Column (Nominal - OneHotEncoding)

from sklearn.preprocessing import OneHotEncoder

# 1. Fit the encoder
ohe = OneHotEncoder(sparse_output=False)
embarked_encoded = ohe.fit_transform(clean_df[['Embarked']])

# 2. Creating a temporary DataFrame with the new columns (e.g., Embarked_C, Embarked_Q, Embarked_S)
ohe_df = pd.DataFrame(embarked_encoded, columns=ohe.get_feature_names_out(['Embarked']))

# 3. Join with clean_df and drop the original column
clean_df = pd.concat([clean_df.reset_index(drop=True), ohe_df], axis=1)
clean_df = clean_df.drop('Embarked', axis=1)

### Selecting Relevant Features

In [97]:
# Keeping useful features and droping not relevant features

clean_df = clean_df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'SibSp', 'Parch'], axis=1)

In [98]:
clean_df.sample(5)

,Survived,Pclass,Age,Fare,Family_size,Family_type,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
519,0,3,32.00,7.8958,1,alone,0,1,0.0,0.0,1.0,0.0
377,0,1,27.00,211.5000,3,small,0,1,1.0,0.0,0.0,0.0
305,1,1,0.92,151.5500,4,small,0,1,0.0,0.0,1.0,0.0
835,1,1,39.00,83.1583,3,small,1,0,1.0,0.0,0.0,0.0
576,1,2,34.00,13.0000,1,alone,1,0,0.0,0.0,1.0,0.0


### Saving Clean CSV File

In [99]:
clean_df.to_csv("./data/clean_data.csv", index=False)